In [ ]:
# step1.프로젝트에 필요한 패키지 불러오기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
# step2.로그인 정보 및 검색할 회사 미리 정의, 해당 회사의 리뷰 끝 페이지도 정의
usr = "hsaurs@gmail.com"
pwd = ""
query = input()
page = 10

In [ ]:
# step3.크롬드라이버 실행 및 잡플래닛 로그인

driver = webdriver.Chrome(chrome_driver)

driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")
time.sleep(5)

login_id = driver.find_element(By.XPATH, '//*[@id="user_email"]')
login_id.send_keys(usr)

login_pwd = driver.find_element(By.XPATH, '//*[@id="user_password"]')
login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)

# step4.원하는 회사의 리뷰 페이지까지 이동
search_query = driver.find_element(By.XPATH, '//*[@id="search_bar_search_query"]')
search_query.send_keys(query)
search_query.send_keys(Keys.RETURN)
time.sleep(3)

driver.find_element(By.XPATH, '//*[@id="mainContents"]/div[1]/div/div[2]/div[1]/div[1]/a').click()
time.sleep(3)

driver.find_element(By.XPATH, '//*[@id="viewCompaniesMenu"]/ul/li[3]/a').click()
time.sleep(15)

driver.find_element(By.CSS_SELECTOR, "button.btn_close_x_ty1 ").click()
time.sleep(3)
  
# step6.크롤링한 정보를 담을 리스트명 정의

list_stars = []
list_merit = []
list_disadvantages = []

# step7.원하는 회사의 평점 및 장단점 크롤링 (for문으로 반복)
for i in range(page): 

    #별점
    stars = driver.find_elements_by_css_selector("div.star_score")
    for j in stars:
        a = j.get_attribute('style')
        if a[7:9] == '20':
            list_stars.append("1점")
        elif a[7:9] == '40':
            list_stars.append("2점")
        elif a[7:9] == '60':
            list_stars.append("3점")
        elif a[7:9] == '80':
            list_stars.append("4점")
        else:
            list_stars.append("5점")
        
    
    #장점, 단점
    list_review = []

    review = driver.find_elements_by_css_selector("dd.df1")

    for j in review:
        list_review.append(j.text)

    for j in range(count):            #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
        a = list_review[3*j]
        list_merit.append(a)
        
        b = list_review[3*j+1]
        list_disadvantages.append(b)

    # 다음 페이지 클릭 후 for문 진행, 끝 페이지에서 다음 페이지 클릭 안되는 것 대비해서 예외처리 구문 추가
    try:
        driver.find_element_by_css_selector("a.btn_pgnext").click()
        time.sleep(15)
    except:
        pass

In [ ]:
# step8.pandas 라이브러리로 표 만들기
total_data = pd.DataFrame()
total_data['별점'] = pd.Series(list_stars)
total_data['장점'] = pd.Series(list_merit)
total_data['단점'] = pd.Series(list_disadvantages)

In [ ]:
total_data.to_excel("코드스테이츠 리뷰 총정리.xlsx" ,index=True)